In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import json

# Load and prepare the dataset
file_path = 'goal_ticks.csv'

# Define column names based on the provided structure
column_names = [
    "kart_ID", "ball_X", "ball_Z", "ball_aim_X", "ball_aim_Z", "ball_node", "previous_X", "previous_Z", "ball_heading",
    "ball_appr_goal", "dist_to_ball", "kart_X", "kart_Z", "vel_X", "vel_Z", "speed", "steer", "accel", "kart_node", 
    "target_encoded", "target_pos_X", "target_pos_Z", "kart0_sector", "kart1_sector", "kart0_X", "kart0_Z", "kart1_X", 
    "kart1_Z", "has_powerup", "goal"
]

# Load the dataset
df = pd.read_csv(file_path, header=None, names=column_names)

# Drop the specified columns
df = df.drop(columns=["target_encoded", "target_pos_X", "target_pos_Z"])

# Extract features and labels
X = df.iloc[:, :-1].values  # All columns except the last one (goal)
y = df['goal'].values  # The 'goal' column as binary

# Normalize all features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler parameters for later use
scaler_params = {
    "mean": scaler.mean_.tolist(),
    "scale": scaler.scale_.tolist()
}

print("Scaler parameters:")
print(json.dumps(scaler_params, indent=4))

with open('scaler_parameters.json', 'w') as f:
    json.dump(scaler_params, f)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Keeping the target as a 2D tensor for BCELoss

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Prepare data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define the neural network architecture
class ScoringModel(nn.Module):
    def __init__(self, num_inputs=26):  # Adjusted number of inputs
        super(ScoringModel, self).__init__()
        self.layer1 = nn.Linear(num_inputs, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, 64)
        self.layer5 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)  # Binary classification
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.leaky_relu(self.layer1(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer2(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer3(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer4(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer5(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x

# Initialize the model with 26 inputs
model = ScoringModel(num_inputs=26)

# Set up the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training function
def train_model(train_loader, model, criterion, optimizer, num_epochs=300):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 10 == 0:  # Print every 10 epochs
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Validation function
def validate_model(val_loader, model, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    print(f'Validation Loss: {total_loss / len(val_loader):.4f}')

# Train and validate the model
train_model(train_loader, model, criterion, optimizer, 300)
validate_model(val_loader, model, criterion)

# Save the trained model for deployment
model.eval()

example_input = torch.rand(1, X_train.shape[1])  # Generate a random example input
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("soccer_ai_model1.pt")

# Print the weights of the output layer after training
print("Weights of the output layer after training:")
print(model.output_layer.weight)
print("Bias of the output layer after training:")
print(model.output_layer.bias)

# Example output
example_output = model(example_input)
print("Example output:", example_output.item())


Scaler parameters:
{
    "mean": [
        0.0,
        0.6645230870132275,
        6.154356330273655,
        0.6808673101380469,
        4.8871244307589965,
        263.21159192325325,
        0.6864503845431057,
        -15.672775404128563,
        0.0014394662891427316,
        0.24834608380402828,
        27.117974125347715,
        0.686553378629333,
        -19.16496224336055,
        0.013211706544853048,
        5.321542929342658,
        6.08604569981789,
        0.014134290888210825,
        0.6280774334248788,
        395.15950144888524,
        395.15950144888524,
        438.09961979904864,
        0.686553378629333,
        -19.16496224336055,
        0.70385851860944,
        29.467038693780548,
        0.1618629846615833
    ],
    "scale": [
        1.0,
        13.269447761412408,
        20.187031418929696,
        13.636423000275762,
        20.74561353927207,
        121.06800614360809,
        12.992114586870263,
        36.58809941209324,
        1.6856073841418